---
title: IDs from Wind
order: 10
---

In [43]:
import speasy as spz
from speasy import SpeasyIndex
from speasy.webservices.generic_archive import user_inventory_dir

In [44]:
spz.update_inventories()

In [45]:
user_inventory_dir()

'/Users/zijin/Library/Application Support/speasy/archive'

In [51]:
data = spz.get_data(
    'archive/local/WI_H4-RTN_MFI/BRTN',
    '2016-01-01',
    '2016-01-03',
)

data.time

array(['2016-01-01T00:00:00.087500000', '2016-01-01T00:00:00.179500000',
       '2016-01-01T00:00:00.271500000', ...,
       '2016-01-02T23:59:59.805500000', '2016-01-02T23:59:59.897500000',
       '2016-01-02T23:59:59.989500000'], dtype='datetime64[ns]')

In [2]:
from discontinuitypy.datasets import IDsDataset
import polars as pl
from fastcore.utils import walk

In [15]:
ts = 1 # unit: seconds
tau = 60 # unit: seconds

mission = "Wind"
data_dir = '../../data'
dir_path = f'{data_dir}/03_primary/{mission}_MAG_ts_{ts}s'
state_data_path = f'{data_dir}/03_primary/OMNI_LowRes_ts_3600s.parquet'
vec_cols = ['v_x', 'v_y', 'v_z']

format = 'arrow'
fname = f'events.{mission}.ts_{ts}s_tau_{tau}s.{format}'
output_path = f'{data_dir}/05_reporting/{fname}'

In [17]:
events = []
for mag_path in files[-1:]:

    mag_data = pl.scan_parquet(mag_path).drop('X', 'Y', 'Z').sort('time')
    plasma_data = pl.scan_parquet(state_data_path).sort('time')

    _events = (
        IDsDataset(
            mag_data=mag_data,
            plasma_data=plasma_data,
            tau=tau,
            ts=ts,
            vec_cols=vec_cols,
        )
        .find_events(return_best_fit=False)
        .update_candidates_with_plasma_data()
        .events
    )
    
    events.append(_events)
    
ids_dataset = IDsDataset(
    events=pl.concat(events),
    mag_data= pl.scan_parquet(list(walk(dir_path))).drop('X', 'Y', 'Z').sort('time')
).export(output_path)

07-Feb-24 16:08:39: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.

07-Feb-24 16:08:40: UserWarning: Ray execution environment not yet initialized. Initializing...
To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()


2024-02-07 16:08:42,624	INFO worker.py:1724 -- Started a local Ray instance.
07-Feb-24 16:08:43: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.



Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

07-Feb-24 16:08:43: Using sequential splitting in '.from_pandas()' because of some of the conditions are False: enough_elements=False; all_numeric_types=False; async_mode_on=False
(_deploy_ray_func pid=71813) UserWarning: Traceback (most recent call last):
(_deploy_ray_func pid=71813)   File "/Users/zijin/micromamba/envs/juno/lib/python3.10/site-packages/pdpipe/__init__.py", line 85, in <module>
(_deploy_ray_func pid=71813)     from . import skintegrate
(_deploy_ray_func pid=71813)   File "/Users/zijin/micromamba/envs/juno/lib/python3.10/site-packages/pdpipe/skintegrate.py", line 20, in <module>
(_deploy_ray_func pid=71813)     from sklearn.base import BaseEstimator
(_deploy_ray_func pid=71813) ModuleNotFoundError: No module named 'sklearn'
(_deploy_ray_func pid=71813) 
(_deploy_ray_func pid=71813) UserWarning: pdpipe: Scikit-learn or skutil import failed. Scikit-learn-dependent pipeline stages will not be loaded.
(_deploy_ray_func pid=71813) UserWarning: Traceback (most recent call la

Estimated completion of line 8:   0%           Elapsed time: 00:00, estimated remaining time: ?

(_deploy_ray_func pid=71812) RuntimeWarning: overflow encountered in exp
(_deploy_ray_func pid=71814)  [repeated 4x across cluster]
(_deploy_ray_func pid=71813) UserWarning: Distributing <class 'dict'> object. This may take some time. [repeated 2145x across cluster]
Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ? [repeated 150x across cluster]
Distributing Dataframe: 100%██████████ Elapsed time: 00:00, estimated remaining time: 00:00 [repeated 1769x across cluster]
Distributing Dataframe: 100%██████████ Elapsed time: 00:00, estimated remaining time: 00:00 [repeated 152x across cluster]
(_deploy_ray_func pid=71813) RuntimeWarning: overflow encountered in exp [repeated 20x across cluster]
(_deploy_ray_func pid=71808) RuntimeWarning: overflow encountered in exp [repeated 25x across cluster]
(_deploy_ray_func pid=71810) RuntimeWarning: overflow encountered in exp [repeated 28x across cluster]


...

07-Feb-24 16:09:20: Using sequential splitting in '.from_pandas()' because of some of the conditions are False: enough_elements=False; all_numeric_types=False; async_mode_on=False


Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

07-Feb-24 16:09:20: Using sequential splitting in '.from_pandas()' because of some of the conditions are False: enough_elements=False; all_numeric_types=True; async_mode_on=False


Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

07-Feb-24 16:09:20: Using sequential splitting in '.from_pandas()' because of some of the conditions are False: enough_elements=False; all_numeric_types=False; async_mode_on=False


## Processing the whole data

## Obsolete

### Estimate

1 day of data resampled by 1 sec is about 12 MB.

So 1 year of data is about 4 GB, and 6 years of JUNO Cruise data is about 24 GB.

Downloading rate is about 250 KB/s, so it will take about 3 days to download all the data.

In [ ]:
num_of_files = 6*365
jno_file_size = 12e3
thm_file_size = 40e3
files_size = jno_file_size + thm_file_size
downloading_rate = 250
processing_rate = 1/60

time_to_download = num_of_files * files_size / downloading_rate / 60 / 60
space_required = num_of_files * files_size / 1e6
time_to_process = num_of_files / processing_rate / 60 / 60

print(f"Time to download: {time_to_download:.2f} hours")
print(f"Disk space required: {space_required:.2f} GB")
print(f"Time to process: {time_to_process:.2f} hours")

Time to download: 126.53 hours
Disk space required: 113.88 GB
Time to process: 36.50 hours
